# Sagemaker Inference

This script predicts new data with the uploaded image in ECR.

## Import modules

In [1]:
import time
import boto3
import sagemaker
from sagemaker import get_execution_role

## Setup

Modify according to your configurations.

In [2]:
# Bucket name in S3
bucket = "hermione-sagemaker"

In [3]:
# Set session
region_name="us-east-1"
boto3.setup_default_session(region_name=region_name)

In [4]:
# Get user role
role = get_execution_role()

In [5]:
# Get AWS Account ID
account_number = boto3.client("sts").get_caller_identity()["Account"]

In [6]:
# Image previous uploaded in ECR
image_name = "hermione-inference"
image_uri = f"{account_number}.dkr.ecr.{region_name}.amazonaws.com/{image_name}"

In [7]:
# Input and output paths to execute inference
paths = {
    'inference_processed': f"s3://{bucket}/PREPROCESSING/INFERENCE_PROCESSED/inference.csv",
    'model': f"s3://{bucket}/PREPROCESSING/MODEL/Hermione-train-2021-05-26-12-41-29-505/output/model.tar.gz",
    'output_path': f"s3://{bucket}/PREPROCESSING/OUTPUT"
}

In [8]:
# instance to run the code
instance_type="ml.m5.large"

## Inference

In [9]:
# Receives the processed inference data in S3
input_path = paths['inference_processed']

In [10]:
# Receives the model created during the training in S3
model_path = paths['model']

In [11]:
# Saves the prediction in S3
output_path = paths['output_path']

In [12]:
# Creates the model to access the ECR image
model = sagemaker.model.Model(
    image_uri= image_uri,
    model_data=model_path,
    role=role)

In [13]:
# Creates a transformer object from the trained model
transformer = model.transformer(
                          instance_count=1,
                          instance_type=instance_type,   
                          output_path=output_path,
                          accept = 'text/csv')

In [14]:
%%time
# Predicts the data
transformer.transform(data=input_path, data_type='S3Prefix', content_type='text/csv', split_type='Line')

2021-07-22 20:28:24,272 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
MMS Home: /usr/local/lib/python3.8/dist-packages
Current directory: /
Temp directory: /tmp
Number of GPUs: 0
Number of CPUs: 2
Max heap size: 1726 M
Python executable: /usr/bin/python3
Config file: /etc/sagemaker-mms.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Model Store: /.sagemaker/mms/models
Initial Models: ALL
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 2
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
Preload model: false
Prefer direct buffer: false
2021-07-22 20:28:24,384 [WARN ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerLifeCycle - attachIOStreams() threadName=W-9000-model
2021-07-22 20:28:24,452 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - model_service_worker started with args: --sock-type unix --sock-name /tmp/.mms.sock.9000 --handler servi